In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import json
from cryptocompy import coin,price
import tqdm
import seaborn as sns
sns.set();

import io
import requests
import json
from datetime import datetime
import time
from time import gmtime, strftime

from sklearn import metrics
from fbprophet import Prophet
plt.rcParams["figure.figsize"] = [14,8]
pd.set_option('float_format', '{:f}'.format)

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import datetime
from datetime import datetime as dt
from datetime import timedelta


from tqdm import tqdm_notebook
from livelossplot.keras import PlotLossesCallback


from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
#from xgboost import XGBRegressor
from catboost import CatBoostRegressor, Pool, cv
import catboost
from sklearn.ensemble import RandomForestRegressor

from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
#from sklearn.cross_validation import cross_val_score
#from sklearn.grid_search import GridSearchCV
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(1)

from keras.models import Sequential  
from keras.layers.core import Dense, Activation  
from keras.layers.recurrent import LSTM
from keras.layers.advanced_activations import LeakyReLU

from keras import optimizers
from keras import backend as K

adam = optimizers.Adam(lr=0.01, clipnorm=1.)

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
def nn_r2_score(y_true, y_pred):
    total_error = K.sum(K.square( y_true - K.mean(y_true) ) )
    residual_error = K.sum(K.square( y_true - y_pred ))
    R_squared = 1 - (residual_error / total_error)
    return -R_squared


from math import sin, cos

from gaft import GAEngine
from gaft.components import BinaryIndividual,DecimalIndividual
from gaft.components import Population
from gaft.operators import TournamentSelection, LinearRankingSelection
from gaft.operators import UniformCrossover
from gaft.operators import FlipBitMutation
from gaft.analysis import ConsoleOutput

# Analysis plugin base class.
from gaft.plugin_interfaces.analysis import OnTheFlyAnalysis

# Built-in best fitness analysis.
from gaft.analysis.fitness_store import FitnessStore

import MySQLdb 
import sshtunnel
from sshtunnel import SSHTunnelForwarder

Using TensorFlow backend.


In [2]:
df = pd.DataFrame()
d1 = pd.DataFrame()
d2 = pd.DataFrame()
d3 = pd.DataFrame()
d4 = pd.DataFrame()
d5 = pd.DataFrame()

d6 = pd.DataFrame()

with SSHTunnelForwarder(('206.189.186.74', 22), ssh_password='crypto1234', ssh_username='aagam', remote_bind_address=('127.0.0.1', 3306)) as server:
    conn = MySQLdb.connect(host='localhost', port=server.local_bind_port, user='ianbalina', passwd='5!sT3jt26K%tFN*W',db='ianbalina')
    cursor = conn.cursor()
    
    #cursor.execute("SELECT * FROM ianbalina.ico_price_daily_summaries;") 
    #m = cursor.fetchone()
    query = '''SELECT * FROM ianbalina.ico_price_daily_summaries;'''
    data = pd.read_sql_query(query, conn)
    df = pd.concat([df,data])
    
    query = '''SELECT * FROM ianbalina.ico_grade_history;'''
    data = pd.read_sql_query(query, conn)
    d1 = pd.concat([d1,data])
    
    query = '''SELECT * FROM ianbalina.ico_analytics;'''
    data = pd.read_sql_query(query, conn)
    d2 = pd.concat([d2,data])
    
    query = '''SELECT * FROM ianbalina.icos;'''
    data = pd.read_sql_query(query, conn)
    d3 = pd.concat([d3,data])
    
    query = '''SELECT * FROM ianbalina.ico_analytics;'''
    data = pd.read_sql_query(query, conn)
    d4 = pd.concat([d4,data])
    
    query = '''SELECT * FROM ianbalina.ico_grade_history_scores;'''
    data = pd.read_sql_query(query, conn)
    d5 = pd.concat([d5,data])
    
    query = '''SELECT * FROM ianbalina.tech_scorecard_options;'''
    data = pd.read_sql_query(query, conn)
    d6 = pd.concat([d6,data])
    
    conn.close() 

I1015 07:02:35.323122 13116 transport.py:1819] Connected (version 2.0, client OpenSSH_7.6p1)
I1015 07:02:37.708897 13116 transport.py:1819] Authentication (password) successful!


In [3]:
def get_bins(v):
    if v <= 10:
        return 0
    elif v > 10 and v <= 20:
        return 1
    elif v > 20 and v <= 30:
        return 2
    elif v > 30 and v <= 40:
        return 3
    elif v > 40 and v <= 50:
        return 4
    elif v > 50 and v <= 60:
        return 5
    elif v > 60 and v <= 70:
        return 6
    elif v > 70 and v <= 80:
        return 7
    elif v > 80 and v <= 90:
        return 8
    elif v > 90:
        return 9

In [4]:
def get_technology_accuracy(weights, max_score, tech):
    t1 = pd.DataFrame()

    for i in tech['ico_symbol'].unique().tolist():
        temp = tech[tech['ico_symbol'] == i]
        temp = temp.sort_values('question_number')
        score = temp['score'] * weights[:len(temp)] 
        
        t1 = pd.concat([t1,pd.DataFrame([i, score.sum() / max_score * 100 / sum(weights[:len(temp)]) * len(temp)]).T])
        
    t1.columns = ['Coin','Score']
    #print(t1)
    #ids = tech['ico_id'][tech['ico_symbol'].isin(t1['Coin'])].unique()
    alpha = []
    for coin in t1['Coin']:
        k = tech['ico_id'][tech['ico_symbol'] == coin].unique()
        alpha.append(d4['alpha'][d4['ico_id'] == k[0]].values[0])
    
    t1['Alpha'] = alpha

    t1['Score_bins'] = t1['Score'].apply(get_bins)
    t1['Alpha_bins'] = pd.cut(t1['Alpha'],[np.floor(t1['Alpha'].min()),0,np.ceil(t1['Alpha'].max())])

    final = t1['Alpha_bins'].groupby([t1['Score_bins'],t1['Alpha_bins']]).count().unstack()

    x0 = []
    x1 = []
    for j in range(10):
        try:
            x0.append(final.loc[j,0])
        except:
            x0.append(0)

    for j in range(10):
        try:
            x1.append(final.loc[j,1])
        except:
            x1.append(0)

    x = pd.DataFrame()
    x[0] = x0
    x[1] = x1
    x.index = ['(0,10]','(10,20]','(20,30]','(30,40]','(40,50]','(50,60]','(60,70]','(70,80]','(80,90]','(90,100]']

    x.fillna(0,inplace=True)

    y = x.sum(axis=1)
    y1 = (x.iloc[0:5,0] / y[:5]) 
    y2 = (x.iloc[5:,1] / y[5:])

    x['Accuracy'] = y1.append(y2).fillna(0).astype('float64').values * 100
    x.columns = [str(final.columns[0]),str(final.columns[1]),'Accuracy']

    accuracy = x.iloc[0:5,0].append(x.iloc[5:10,1]).sum() / x.iloc[:,:2].sum(axis=1).sum()
    return x, accuracy

In [5]:
tech = d5[d5['type'] == 1]
d1 = d1[d1['date'] == d1['date'].sort_values().iloc[-1]]
tech = tech[tech['ico_grade_history_id'].isin(d1['id'].unique())]
tech = pd.merge(tech,d1[['id','ico_id']], left_on = 'ico_grade_history_id', right_on = 'id', how='left')
tech = tech.drop('id_y', axis=1)
tech.rename(columns={'id_x':'id'}, inplace=True)

ids = d3['id'][d3['fundamentals_published_at'].notnull()].unique().tolist()
coin = []
for id in ids:
    coin.append(df['ico_symbol'][df['ico_id'] == id].unique().tolist()[0])
    
tech = tech[tech['ico_id'].isin(ids)]
temp = pd.DataFrame([ids,coin]).T
temp.columns = ['ico_id','ico_symbol']

tech = pd.merge(tech,temp)
tech['question_number'] = tech['name'].str.split('_').str.get(-1)

In [16]:
weights = np.ones(28)
max_score = 100#d6['score'].groupby([d6['tech_scorecard_question_id']]).max().sum()

get_technology_accuracy(weights, max_score, tech)

    Coin     Score
0    TRX 59.929900
0    ICX 64.787300
0    ONT 63.664900
0    ETH 90.304000
0    NEO 76.081700
0    BNB 77.581700
0    ADA 72.728000
0    LSK 70.719600
0    REP 78.840000
0   QTUM 73.214900
0    OMG 78.995000
0    BAT 84.110200
0    EOS 63.665500
0    XRP 57.848100
0    BTC 93.626000
0    BCH 79.085000
0    LTC 74.330000
0    XLM 73.315200
0    XMR 86.896000
0   DASH 84.660000
0    BSV 73.728000
0    XTZ 71.634800
0  MIOTA 72.342300
0    ETC 71.294000
0    MKR 81.114000
0    ZEC 86.575000
0   DOGE 53.304000
0    BTG 63.347000
0    DCR 82.650500
0    BTT 48.928200
0     HT 71.009800
0    KCS 48.654200
0   ATOM 81.771500
0    VET 81.762700
0   VSYS 52.853400


(          (-142.0, 0.0]  (0.0, 92.0]   Accuracy
 (0,10]         0.000000     0.000000   0.000000
 (10,20]        0.000000     0.000000   0.000000
 (20,30]        0.000000     0.000000   0.000000
 (30,40]        0.000000     0.000000   0.000000
 (40,50]        1.000000     1.000000  50.000000
 (50,60]        4.000000     0.000000   0.000000
 (60,70]        2.000000     2.000000  50.000000
 (70,80]       10.000000     5.000000  33.333333
 (80,90]        5.000000     3.000000  37.500000
 (90,100]       0.000000     2.000000 100.000000, 0.37142857142857144)

In [7]:
d1 = d1[d1['date'] == d1['date'].sort_values().iloc[-1]]

In [8]:
d4['alpha'][d4['ico_id'].isin(ids)].values

array([ -41.0215,   -8.7302,   -5.7908,   -0.7861,  -11.1544,  -11.0794,
         11.0396,    0.3483,  -12.4628,   -5.8929,   -6.0445,   -0.2176,
         -2.8531,    6.1803,   -3.1604,    2.1257,   -0.5533,   24.3788,
          0.143 ,    8.9078,  -12.38  ,    9.3292,    9.5223,  -15.0097,
        -10.7704,   45.8284,   -2.5535,   -2.4369,    6.6354,  -10.917 ,
          2.1384, -141.8958,  -73.9418,  -12.5754,   91.9429])

In [9]:
ids = d3['id'][d3['fundamentals_published_at'].notnull()].unique().tolist()
coin = []
for id in ids:
    coin.append(df['ico_symbol'][df['ico_id'] == id].unique().tolist()[0])
    
temp = pd.DataFrame()
temp['Coin'] = coin
temp['Score'] = d1['tokenmetrics_score'][d1['ico_id'].isin(ids)].values
temp['Alpha'] = d4['alpha'][d4['ico_id'].isin(ids)].values

In [10]:
temp[['Score','Alpha']] = temp[['Score','Alpha']].astype('float64')
'Correlation: ', temp.corr().iloc[0,1]

('Correlation: ', 0.1620055348224786)

In [11]:
temp

,Coin,Score,Alpha
0,TRX,59.929900,-41.021500
1,ICX,64.787300,-8.730200
2,ONT,63.664900,-5.790800
3,ETH,90.304000,-0.786100
4,NEO,76.081700,-11.154400
5,BNB,77.581700,-11.079400
6,ADA,72.728000,11.039600
7,LSK,70.719600,0.348300
8,REP,78.840000,-12.462800
9,QTUM,73.214900,-5.892900


In [17]:
temp['Score_bins'] = pd.cut(temp['Score'],[0,10,20,30,40,50,60,70,80,90,100])
temp['Alpha_bins'] = pd.cut(temp['Alpha'],[np.floor(temp['Alpha'].min()),0,np.ceil(temp['Alpha'].max())])

In [18]:
final = temp['Alpha_bins'].groupby([temp['Score_bins'],temp['Alpha_bins']]).count().unstack()
final

Alpha_bins  (-142.0, 0.0]  (0.0, 92.0]
Score_bins                            
(40, 50]         2.000000          nan
(50, 60]         2.000000     2.000000
(60, 70]         4.000000          nan
(70, 80]         8.000000     7.000000
(80, 90]         4.000000     4.000000
(90, 100]        2.000000          nan

In [19]:
x = pd.DataFrame()
x[0] = [0,0,0,0] + final.values[:,0].tolist()
x[1] = [0,0,0,0] + final.values[:,1].tolist()
x.index = ['(0,10]','(10,20]','(20,30]','(30,40]','(40,50]','(50,60]','(60,70]','(70,80]','(80,90]','(90,100]']

x.fillna(0,inplace=True)
#x[final.columns[0]] = x[final.columns[0]].astype('float64')
#x[final.columns[1]] = x[final.columns[1]].astype('float64')

y = x.sum(axis=1)
y1 = (x.iloc[0:5,0] / y[:5]) 
y2 = (x.iloc[5:,1] / y[5:])

x['Accuracy'] = y1.append(y2).fillna(0).astype('float64').values * 100
x.columns = [str(final.columns[0]),str(final.columns[1]),'Accuracy']

In [20]:
x

,"(-142.0, 0.0]","(0.0, 92.0]",Accuracy
"(0,10]",0.000000,0.000000,0.000000
"(10,20]",0.000000,0.000000,0.000000
"(20,30]",0.000000,0.000000,0.000000
"(30,40]",0.000000,0.000000,0.000000
"(40,50]",2.000000,0.000000,100.000000
"(50,60]",2.000000,2.000000,50.000000
"(60,70]",4.000000,0.000000,0.000000
"(70,80]",8.000000,7.000000,46.666667
"(80,90]",4.000000,4.000000,50.000000
"(90,100]",2.000000,0.000000,0.000000


In [21]:
x.iloc[0:5,0].append(x.iloc[5:10,1]).sum() / x.iloc[:,:2].sum(axis=1).sum()

0.42857142857142855

In [24]:
indv_template = DecimalIndividual(ranges=[(0,10)]*28, eps=[1]*28)
population = Population(indv_template=indv_template, size=100).init()

selection = LinearRankingSelection()
crossover = UniformCrossover(pc=0.8, pe=0.5)
mutation = FlipBitMutation(pm=0.1)

# Create genetic algorithm engine.
engine = GAEngine(population=population, selection=selection,
                  crossover=crossover, mutation=mutation,
                  analysis=[FitnessStore])

@engine.fitness_register
#@engine.minimize
def fitness(indv):
    w = list(indv.solution)
    _, res = get_technology_accuracy(w, max_score, tech)
    res = float(abs(res))
    #print(w,': ',res)
    return res

def get_test_results(w1,w2,w3,w4):
    predictions = m1*w1 + m2*w2 + m3*w3 + m4*w4
    print('\nRMSE error: ', np.sqrt(metrics.mean_squared_error(Y_test, predictions)))
    for i,prediction in enumerate(predictions[:15]):
        print ('Target: %s, Predicted: %s' % (Y_test.tolist()[i], abs(prediction)))

# Define on-the-fly analysis.
@engine.analysis_register
class ConsoleOutputAnalysis(OnTheFlyAnalysis):
    interval = 1
    master_only = True

    def register_step(self, g, population, engine):
        best_indv = population.best_indv(engine.fitness)
        msg = 'Generation: {}, best fitness: {:.3f}, best_individual: {}'.format(g, engine.ori_fmax, best_indv.solution)
        self.logger.info(msg)

    def finalize(self, population, engine):
        best_indv = population.best_indv(engine.fitness)
        x = best_indv.solution
        y = engine.fmax
        msg = 'Optimal solution: ({}, {})'.format(x, y)
        self.logger.info(msg)

if '__main__' == __name__:
    # Run the GA engine.
    engine.run(ng=100)
    best_indv = engine.population.best_indv(engine.fitness)
    w1,w2,w3,w4 = best_indv.solution
    get_test_results(w1,w2,w3,w4)

gaft.ConsoleOutputAnalysis   INFO     Generation: 0, best fitness: 0.543, best_individual: [7, 10, 0, 5, 4, 8, 1, 2, 6, 0, 2, 10, 9, 7, 6, 10, 7, 1, 6, 0, 6, 5, 1, 5, 8, 0, 6, 6]


I1014 08:12:52.710677 11872 <ipython-input-24-46d7ad2432ef>:37] Generation: 0, best fitness: 0.543, best_individual: [7, 10, 0, 5, 4, 8, 1, 2, 6, 0, 2, 10, 9, 7, 6, 10, 7, 1, 6, 0, 6, 5, 1, 5, 8, 0, 6, 6]


gaft.ConsoleOutputAnalysis   INFO     Generation: 1, best fitness: 0.543, best_individual: [7, 10, 0, 5, 4, 8, 1, 2, 6, 0, 2, 10, 9, 7, 6, 10, 7, 1, 6, 0, 6, 5, 1, 5, 8, 0, 6, 6]


I1014 08:13:37.040265 11872 <ipython-input-24-46d7ad2432ef>:37] Generation: 1, best fitness: 0.543, best_individual: [7, 10, 0, 5, 4, 8, 1, 2, 6, 0, 2, 10, 9, 7, 6, 10, 7, 1, 6, 0, 6, 5, 1, 5, 8, 0, 6, 6]


gaft.ConsoleOutputAnalysis   INFO     Generation: 2, best fitness: 0.543, best_individual: [7, 10, 0, 5, 4, 8, 1, 2, 6, 0, 2, 10, 9, 7, 6, 10, 7, 1, 6, 0, 6, 5, 1, 5, 8, 0, 6, 6]


...

gaft.FitnessStore   INFO     Best fitness values are written to best_fit.py


I1014 09:24:58.762840 11872 fitness_store.py:48] Best fitness values are written to best_fit.py


gaft.ConsoleOutputAnalysis   INFO     Optimal solution: ([5, 2, 5, 7, 3, 0, 1, 0, 4, 1, 10, 7, 8, 6, 8, 10, 9, 0, 1, 0, 2, 7, 1, 2, 3, 8, 1, 7], 0.6)


I1014 09:24:58.767853 11872 <ipython-input-24-46d7ad2432ef>:44] Optimal solution: ([5, 2, 5, 7, 3, 0, 1, 0, 4, 1, 10, 7, 8, 6, 8, 10, 9, 0, 1, 0, 2, 7, 1, 2, 3, 8, 1, 7], 0.6)


ValueError: too many values to unpack (expected 4)

In [25]:
weights = [5, 2, 5, 7, 3, 0, 1, 0, 4, 1, 10, 7, 8, 6, 8, 10, 9, 0, 1, 0, 2, 7, 1, 2, 3, 8, 1, 7]

max_score = 100#d6['score'].groupby([d6['tech_scorecard_question_id']]).max().sum()

y,z = get_technology_accuracy(weights, max_score, tech)

In [26]:
y

,"(-142.0, 0.0]","(0.0, 92.0]",Accuracy
"(0,10]",0,0,0.000000
"(10,20]",0,0,0.000000
"(20,30]",0,0,0.000000
"(30,40]",2,1,66.666667
"(40,50]",8,1,88.888889
"(50,60]",7,4,36.363636
"(60,70]",4,5,55.555556
"(70,80]",1,2,66.666667
"(80,90]",0,0,0.000000
"(90,100]",0,0,0.000000


In [28]:
x

,"(-142.0, 0.0]","(0.0, 92.0]",Accuracy
"(0,10]",0.000000,0.000000,0.000000
"(10,20]",0.000000,0.000000,0.000000
"(20,30]",0.000000,0.000000,0.000000
"(30,40]",0.000000,0.000000,0.000000
"(40,50]",2.000000,0.000000,100.000000
"(50,60]",2.000000,2.000000,50.000000
"(60,70]",4.000000,0.000000,0.000000
"(70,80]",8.000000,7.000000,46.666667
"(80,90]",4.000000,4.000000,50.000000
"(90,100]",2.000000,0.000000,0.000000
